In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import os

class RetinopathyDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            image = plt.imread(self.image_paths[idx])
            label = self.labels[idx]
            
            if image.ndim == 2:  # Grayscale
                image = np.stack([image] * 3, axis=-1)
            if image.shape[-1] != 3:  # Ensure 3 channels
                image = np.stack([image[..., 0]] * 3, axis=-1)
            
            if self.transform:
                image = self.transform(image)
                
            return image, label
        except Exception as e:
            print(f"Error loading image {self.image_paths[idx]}: {e}")
            return torch.zeros((3, 224, 224)), self.labels[idx]

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class RetinopathyModel(nn.Module):
    def __init__(self, num_classes=5):
        super(RetinopathyModel, self).__init__()
        self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    early_stopping = EarlyStopping(patience=patience)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_bar.set_postfix({'loss': running_loss/len(train_loader)})
        
        epoch_train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        
        val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
        with torch.no_grad():
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                all_preds.extend(outputs.softmax(dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                val_bar.set_postfix({'loss': val_loss/len(val_loader)})
        
        epoch_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        
        scheduler.step(epoch_val_loss)
        early_stopping(epoch_val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    plot_metrics(all_labels, all_preds, train_losses, val_losses, train_accs, val_accs)
    print_model_summary(all_labels, all_preds, train_losses, val_losses, train_accs, val_accs, epoch + 1)
    
    return model

def test_model(model_path, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RetinopathyModel(num_classes=5)
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model = model.to(device)
    model.eval()
    
    test_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    criterion = nn.CrossEntropyLoss()
    
    test_bar = tqdm(test_loader, desc='Testing')
    with torch.no_grad():
        for inputs, labels in test_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(outputs.softmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            test_bar.set_postfix({'loss': test_loss/len(test_loader)})
    
    test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * correct / total
    
    print("\n=== Test Results ===")
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy:.2f}%')
    
    pred_labels = np.argmax(all_preds, axis=1)
    report = classification_report(all_labels, pred_labels, 
                                 target_names=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'])
    
    cm = confusion_matrix(all_labels, pred_labels)
    cm_df = pd.DataFrame(cm, 
                        index=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'],
                        columns=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'])
    
    print("\nTest Classification Report:")
    print(report)
    print("\nTest Confusion Matrix:")
    print(cm_df)
    
    plt.figure(figsize=(8, 6))
    for i in range(5):
        fpr, tpr, _ = roc_curve(np.array(all_labels) == i, np.array(all_preds)[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Test ROC Curve')
    plt.legend()
    plt.show()

def plot_metrics(labels, preds, train_losses, val_losses, train_accs, val_accs):
    fpr = {}
    tpr = {}
    roc_auc = {}
    n_classes = 5
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(np.array(labels) == i, np.array(preds)[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    
    plt.subplot(1, 3, 2)
    cm = confusion_matrix(labels, np.argmax(preds, axis=1))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    plt.subplot(1, 3, 3)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.plot(train_accs, label='Train Acc')
    plt.plot(val_accs, label='Val Acc')
    plt.title('Training Metrics')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def print_model_summary(labels, preds, train_losses, val_losses, train_accs, val_accs, epochs_run):
    print("\n=== Model Performance Summary ===")
    
    final_train_loss = train_losses[-1]
    final_val_loss = val_losses[-1]
    final_train_acc = train_accs[-1]
    final_val_acc = val_accs[-1]
    
    pred_labels = np.argmax(preds, axis=1)
    report = classification_report(labels, pred_labels, 
                                 target_names=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'])
    
    roc_auc_scores = {}
    for i in range(5):
        fpr, tpr, _ = roc_curve(np.array(labels) == i, np.array(preds)[:, i])
        roc_auc_scores[f'Class {i}'] = auc(fpr, tpr)
    
    summary_data = {
        'Metric': ['Epochs Run', 'Final Train Loss', 'Final Val Loss', 'Final Train Accuracy', 'Final Val Accuracy'],
        'Value': [epochs_run, f'{final_train_loss:.4f}', f'{final_val_loss:.4f}', 
                 f'{final_train_acc:.2f}%', f'{final_val_acc:.2f}%']
    }
    summary_df = pd.DataFrame(summary_data)
    
    print("\nGeneral Metrics:")
    print(summary_df.to_string(index=False))
    
    print("\nROC AUC Scores:")
    for class_name, score in roc_auc_scores.items():
        print(f"{class_name}: {score:.4f}")
        
    print("\nDetailed Classification Report:")
    print(report)
    
    cm = confusion_matrix(labels, pred_labels)
    cm_df = pd.DataFrame(cm, 
                        index=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'],
                        columns=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'])
    print("\nConfusion Matrix:")
    print(cm_df)

def load_data_from_folders(root_path):
    root_path = Path(root_path)
    splits = ['train', 'val', 'test']
    class_dirs = ['0', '1', '2', '3', '4']
    
    datasets = {}
    
    for split in splits:
        image_paths = []
        labels = []
        
        split_path = root_path / split
        if not split_path.exists():
            print(f"Warning: {split_path} does not exist")
            continue
            
        print(f"\nProcessing {split} split:")
        for class_dir in class_dirs:
            class_path = split_path / class_dir
            if not class_path.exists():
                print(f"Warning: {class_path} does not exist")
                continue
                
            class_label = int(class_dir)
            class_images = list(class_path.glob("*.[jp][pn][eg]"))
            
            print(f"Class {class_label}: Found {len(class_images)} images")
            
            image_paths.extend(class_images)
            labels.extend([class_label] * len(class_images))
        
        if not image_paths:
            raise ValueError(f"No images found in {split} split")
            
        print(f"Total images in {split}: {len(image_paths)}")
        label_counts = pd.Series(labels).value_counts().sort_index()
        print(f"Label distribution in {split}:")
        print(label_counts)
        
        datasets[split] = (image_paths, labels)
    
    return datasets

def main():
    root_path = "/kaggle/input/eyepacs-aptos-messidor-diabetic-retinopathy/augmented_resized_V2"
    
    try:
        datasets = load_data_from_folders(root_path)
    except ValueError as e:
        print(f"Error loading data: {e}")
        return
    
    train_dataset = RetinopathyDataset(
        datasets['train'][0], 
        datasets['train'][1], 
        transform=data_transforms['train']
    )
    val_dataset = RetinopathyDataset(
        datasets['val'][0], 
        datasets['val'][1], 
        transform=data_transforms['val']
    )
    test_dataset = RetinopathyDataset(
        datasets['test'][0], 
        datasets['test'][1], 
        transform=data_transforms['test']
    )
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
    
    labels = datasets['train'][1]
    class_counts = pd.Series(labels).value_counts().sort_index()
    total_samples = len(labels)
    num_classes = 5
    class_weights = torch.tensor([total_samples / (num_classes * class_counts[i]) for i in range(num_classes)], 
                                dtype=torch.float).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    
    model = RetinopathyModel(num_classes=5)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    trained_model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=50,
        patience=7
    )
    
    model_path = 'retinopathy_model.pth'
    torch.save(trained_model.state_dict(), model_path)
    
    test_model(model_path, test_loader)

if __name__ == "__main__":
    main()